# Solving Sudoku Puzzles


### Choice of Algorithm

The implementation includes a backtracking depth-first search with constraint propagation.


If you choose to implement more than one algorithm, please feel free to include your code and write about it in part two (report), but only the code in this notebook will be used in the automated testing.

## Sample Sudoku Puzzles
This is a sample sudoku puzzle. Sudoku's are differentiated by diffuculty. Not all sudoku puzzles have a solution.

Each sudoku is a 9x9 NumPy array of integers, where zero represents an empty square. Each difficulty comes with 15 sudokus, so when the file is loaded, it is stored in a 15x9x9 array.

In [1]:
import numpy as np

# Load sudokus
sudoku = np.load("data/very_easy_puzzle.npy")
print("very_easy_puzzle.npy has been loaded into the variable sudoku")
print(f"sudoku.shape: {sudoku.shape}, sudoku[0].shape: {sudoku[0].shape}, sudoku.dtype: {sudoku.dtype}")

# Load solutions for demonstration
solutions = np.load("data/very_easy_solution.npy")
print()

# Print the first 9x9 sudoku...
print("First sudoku:")
print(sudoku[0], "\n")

# ...and its solution
print("Solution of first sudoku:")
print(solutions[0])

very_easy_puzzle.npy has been loaded into the variable sudoku
sudoku.shape: (15, 9, 9), sudoku[0].shape: (9, 9), sudoku.dtype: int8

First sudoku:
[[1 0 4 3 8 2 9 5 6]
 [2 0 5 4 6 7 1 3 8]
 [3 8 6 9 5 1 4 0 2]
 [4 6 1 5 2 3 8 9 7]
 [7 3 8 1 4 9 6 2 5]
 [9 5 2 8 7 6 3 1 4]
 [5 2 9 6 3 4 7 8 1]
 [6 0 7 2 9 8 5 4 3]
 [8 4 3 0 1 5 2 6 9]] 

Solution of first sudoku:
[[1 7 4 3 8 2 9 5 6]
 [2 9 5 4 6 7 1 3 8]
 [3 8 6 9 5 1 4 7 2]
 [4 6 1 5 2 3 8 9 7]
 [7 3 8 1 4 9 6 2 5]
 [9 5 2 8 7 6 3 1 4]
 [5 2 9 6 3 4 7 8 1]
 [6 1 7 2 9 8 5 4 3]
 [8 4 3 7 1 5 2 6 9]]


## Part One

A function named `sudoku_solver(sudoku)` is included hich takes one sudoku puzzle (a 9x9 NumPy array) as input, and returns the solved sudoku as another 9x9 NumPy array.

In [2]:
import numpy as np
import random
import copy

class PartialSudokuState:
    """
    Initializing the Partial Sudoku state class with a grid variable,
    which deepcopies the sudoku that needs to be solved and a 
    possible_values array for each cell(square) on the grid
    """
    def __init__(self, sudoku):
        self.grid = copy.deepcopy(sudoku)
        self.possible_values = [[([1,2,3,4,5,6,7,8,9] if self.grid[i][j] == 0 else []) for j in range(9)] for i in range(9)]  
       
    def is_goal(self):
        """
        The function checks whether the partial state is a goal state if
        the grid has no empty cells(squares) and all the rows, columns and 3x3 subgrids
        are valid
        """
        return all(self.grid[i][j] != 0 for i in range(9) for j in range(9)) and \
            not self.is_invalid_row() and \
            not self.is_invalid_column() and \
            not self.is_invalid_subgrid()
       
    def is_invalid(self):
        """
        The function checks whether the current state of the grid is invalid by
        checking whether any of the rows, columns and 3x3 subgrids are invalid and whether
        there are no possible values for all cells(squares) on the grid
        """
        
        return self.has_no_options() or \
            self.is_invalid_row() or \
            self.is_invalid_column() or \
            self.is_invalid_subgrid()

    def has_no_options(self):
        """
        The function checks whether there are no possible values for all cells(squares) on the grid
        """
        for row_values in self.possible_values:
            for cell_values in row_values:
                if len(cell_values) > 0:
                    return False
        return True
   
    def is_invalid_row(self):
        """
        The function checks whethere any of the rows on the sudoku grid are invalid.
        It uses a filtered list to take into account the 0's which act as non-filled
        cells(squares) and that they are not an ordinary 1 to 9 number.
        """
        for row in range(self.grid.shape[0]):
            cur_row = self.grid[row, :]
           
            filtered_list = [value for value in cur_row if value != 0]
           
            if len(np.unique(filtered_list)) != len(filtered_list):
                return True
        return False

    def is_invalid_column(self):
        """
        The function checks whethere any of the columns on the sudoku grid are invalid.
        It uses a filtered list to take into account the 0's which act as non-filled
        cells(squares) and that they are not an ordinary 1 to 9 number.
        """
        for col in range(self.grid.shape[1]):
            cur_column = self.grid[:, col]
               
            filtered_list = [value for value in cur_column if value != 0]
           
            if len(np.unique(filtered_list)) != len(filtered_list):
                return True
        return False

    def is_invalid_subgrid(self):
        """
        The function checks whethere any of the 3x3 subgrids on the sudoku grid are invalid.
        It uses a filtered list to take into account the 0's which act as non-filled
        cells(squares) and that they are not an ordinary 1 to 9 number.
        """
        for i in range(9):
            subgrid = self.grid[(i//3)*3:(i//3)*3+3, (i%3)*3:(i%3)*3+3]
            flat_subgrid = subgrid.flatten()
           
            filtered_list = [value for value in flat_subgrid if value != 0]
       
            if len(np.unique(filtered_list)) != len(filtered_list):
                return True
   
        return False

    def get_possible_values(self, row, column):
        """
        The function gets the possible values for a given cell(square)
        """
        return self.possible_values[row][column].copy()
   
    def get_singleton_squares(self):
        """
        The function returns the cells(squares) which do not have final value, but
        have only 1 possible value
        """
        singleton_squares = []
        for row in range(9):
            for column in range(9):
                if len(self.possible_values[row][column]) == 1 and self.grid[row][column] == 0:
                    singleton_squares.append((row, column))
        return singleton_squares
           
    def set_value(self, row, column, value):
        """
        The function returns a new state with the square set to the value parameter
        in a specific cell(square) that sits on the row and columns variables and 
        then propagates the change to the other squares
        """
        if value not in self.possible_values[row][column]:
            raise ValueError(f"{value} is not a valid choice for this square ({row}, {column})")
       
        # the method returns a new state without modifying the existing one by creating a deepcopy
        state = copy.deepcopy(self)
       
        # update this cell(square)
        state.grid[row][column] = value
        state.possible_values[row][column] = []
       
       
        # now update all other cells(squares) possible values
        # updating the row and the column possible values
        for i in range(9):
            if value in state.possible_values[row][i]:
                state.possible_values[row][i].remove(value)
            if value in state.possible_values[i][column]:    
                state.possible_values[i][column].remove(value)
           
        # updating the 3x3 subgrid possible values
        subgrid_start_row = 3 * (row // 3)
        subgrid_start_column = 3 * (column // 3)
        for x in range(subgrid_start_row, subgrid_start_row + 3):
            for y in range(subgrid_start_column, subgrid_start_column + 3):
                if value in state.possible_values[x][y]:
                    state.possible_values[x][y].remove(value)
       
        # updating any other cells(squares) that have no final value but have only 1 possible value
        singleton_squares = state.get_singleton_squares()
        while len(singleton_squares) > 0:
            square_row, square_column = singleton_squares[0]
            state = state.set_value(square_row, square_column, state.possible_values[square_row][square_column][0])
            singleton_squares = state.get_singleton_squares()
       
        return state

def pick_next_cell(partial_state):
    """
    The function is used in depth-first search. It chooses a random
    cell(square) that has more than one possible value
    """
    cells = []
    for row in range(9):
        for column in range(9):
            if (len(partial_state.get_possible_values(row, column)) > 0):
                cells.append((row, column))
               
    return random.choice(cells)

def order_values(partial_state, row, column):
    """
    The function gets the possible values for a particular cell(square)
    in the order we should try them in. It shuffles them in a random order.
    """
    values = partial_state.get_possible_values(row, column)
    random.shuffle(values)
    return values


def depth_first_search(partial_state):
    """
    The function will execute a depth-first search on the partial states, trying
    all of the possible value for a cell(square).

    The function may try all the possible values for a single cell(square) and
    if it does not find a solution, it implies that there are not possible values
    for at least one of the cells(squares) on the grid and therefore the sudoku has no
    solution.
    """
    row, column = pick_next_cell(partial_state)
    values = order_values(partial_state, row, column)

    for value in values:
        new_state = partial_state.set_value(row, column, value)
       
        if new_state.is_goal():
            return new_state
        if not new_state.is_invalid():
            deep_state = depth_first_search(new_state)
            if deep_state is not None and deep_state.is_goal():
                return deep_state
           
    return None

def sudoku_solver(sudoku):
    """
    The function, which takes the given sudoku that needs to be solved as
    input and performs the depth_first_search algorithm on it. If it
    finds a solution, it returns the solved sudoku, otherwise it returns
    a sudoku with -1 on each cell(square) on the grid.
    """
    initial_state = PartialSudokuState(sudoku)
    solution_state = depth_first_search(initial_state)
   
    if solution_state != None and solution_state.is_goal():
        return np.array(solution_state.grid)
    else:
        return np.full((9, 9), -1)



### Testing Details

In case a certain sudoku has no solution, a 9x9 NumPy array whose values are all equal to -1 should be returned.

#### Test Cell

In [3]:
SKIP_TESTS = True

if not SKIP_TESTS:
    import time
    import numpy as np
    __SCORES = {}
    difficulties = ['very_easy', 'easy', 'medium', 'hard']

    for difficulty in difficulties:
        print(f"Testing {difficulty} sudokus")
        
        sudokus = np.load(f"data/{difficulty}_puzzle.npy")
        solutions = np.load(f"data/{difficulty}_solution.npy")
        
        count = 0
        for i in range(len(sudokus)):
            sudoku = sudokus[i].copy()
            print(f"This is {difficulty} sudoku number", i)
            print(sudoku)
            
            start_time = time.process_time()
            your_solution = sudoku_solver(sudoku)
            end_time = time.process_time()
            
            if not isinstance(your_solution, np.ndarray):
                print("\033[91m[ERROR] Your sudoku_solver function returned a variable that has the incorrect type. If you submit this it will likely fail the auto-marking procedure result in a mark of 0 as it is expecting the function to return a numpy array with a shape (9,9).\n\t\033[94mYour function returns a {} object when {} was expected.\n\x1b[m".format(type(your_solution), np.ndarray))
            elif not np.all(your_solution.shape == (9, 9)):
                print("\033[91m[ERROR] Your sudoku_solver function returned an array that has the incorrect shape.  If you submit this it will likely fail the auto-marking procedure result in a mark of 0 as it is expecting the function to return a numpy array with a shape (9,9).\n\t\033[94mYour function returns an array with shape {} when {} was expected.\n\x1b[m".format(your_solution.shape, (9, 9)))
            
            print(f"This is your solution for {difficulty} sudoku number", i)
            print(your_solution)
            
            print("Is your solution correct?")
            if np.array_equal(your_solution, solutions[i]):
                print("Yes! Correct solution.")
                count += 1
            else:
                print("No, the correct solution is:")
                print(solutions[i])
            
            print("This sudoku took {} seconds to solve.\n".format(end_time-start_time))

        print(f"{count}/{len(sudokus)} {difficulty} sudokus correct")
        __SCORES[difficulty] = {
            'correct': count,
            'total': len(sudokus)
        }